In [19]:
import requests 
import pandas as pd
from datetime import datetime,date 

In [20]:
def fetch_crypto_data(crypto_pair: str, start_date: str) -> pd.DataFrame:
    """
    Fetches historical cryptocurrency data from the CoinGecko API.
    
    Parameters:
    - crypto_pair (str): The cryptocurrency pair to fetch data for (e.g., "bitcoin/usd").
    - start_date (str): The start date for fetching historical data in 'YYYY-MM-DD' format.
    
    Returns:
    - pd.DataFrame: A DataFrame containing historical price data with columns: Date, Open, High, Low, Close.
    """
    
    # Split the crypto_pair into its components: crypto_id and currency
    crypto_id, currency = crypto_pair.split('/')
    
    # Convert the start date to a UNIX timestamp
    start_timestamp = int(datetime.strptime(start_date, '%Y-%m-%d').timestamp())
    
    # Construct the API URL for fetching market chart data within a specific date range
    url = f"https://api.coingecko.com/api/v3/coins/{crypto_id}/market_chart/range"
    
    # Set up parameters for the API request
    params = {
        'vs_currency': currency.lower(),  # The currency to compare against (e.g., 'usd')
        'from': start_timestamp,           # Start timestamp
        'to': int(datetime.now().timestamp()),  # Current timestamp
    }
    
    # Send a GET request to the CoinGecko API
    response = requests.get(url, params=params)
    
    # Check if the response status is not OK (200)
    if response.status_code != 200:
        raise Exception(f"Error fetching data: {response.status_code} - {response.text}")

    # Parse the JSON response
    data = response.json()

    # Check if 'prices' are present in the response data
    if 'prices' not in data:
        raise Exception("No price data found in the response.")

    # Create a DataFrame from the price data (timestamps and closing prices)
    df = pd.DataFrame(data['prices'], columns=['timestamp', 'close'])
    
    # Convert the timestamp from milliseconds to a date format and add it to the DataFrame
    df['Date'] = pd.to_datetime(df['timestamp'], unit='ms').dt.date
    
    # Initialize Open, High, Low, and Close columns with closing prices
    df['Open'] = df['close']
    df['High'] = df['close']
    df['Low'] = df['close']
    df['Close'] = df['close']

    # Group the DataFrame by date and aggregate the Open, High, Low, and Close values
    df = df.groupby('Date').agg({
        'Open': 'first',  # First entry for Open price
        'High': 'max',    # Maximum price for High
        'Low': 'min',     # Minimum price for Low
        'Close': 'last'   # Last entry for Close price
    }).reset_index()

    return df

# Example usage
if __name__ == "__main__":
    crypto_pair = "bitcoin/usd"
    
    from datetime import timedelta
    start_date = (datetime.today() - timedelta(days=360)).strftime('%Y-%m-%d')

    try:
        crypto_data = fetch_crypto_data(crypto_pair, start_date)
        print(crypto_data)
    except Exception as e:
        print(e)


           Date           Open           High            Low          Close
0    2024-07-07   58230.484154   58230.484154   58230.484154   58230.484154
1    2024-07-08   55880.375378   55880.375378   55880.375378   55880.375378
2    2024-07-09   56665.140734   56665.140734   56665.140734   56665.140734
3    2024-07-10   57988.303512   57988.303512   57988.303512   57988.303512
4    2024-07-11   57704.202284   57704.202284   57704.202284   57704.202284
..          ...            ...            ...            ...            ...
356  2025-06-28  107078.915606  107078.915606  107078.915606  107078.915606
357  2025-06-29  107331.585485  107331.585485  107331.585485  107331.585485
358  2025-06-30  108396.616313  108396.616313  108396.616313  108396.616313
359  2025-07-01  107132.799107  107132.799107  107132.799107  107132.799107
360  2025-07-02  105613.399742  105613.399742  105613.399742  105613.399742

[361 rows x 5 columns]
